**베이스라인 코드 (LB : 0.79167)**
with dense layer model / tensorflow / keras

**필요한 라이브러리 불러오기**

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

ModuleNotFoundError: No module named 'tensorflow'

**데이터 불러오기**

In [ ]:
train      = pd.read_csv("./open/train_data.csv")
test       = pd.read_csv("./open/test_data.csv")
submission = pd.read_csv("./open/sample_submission.csv")
topic_dict = pd.read_csv("./open/topic_dict.csv")

**데이터 전처리**

In [ ]:
def clean_text(sent):
  sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
  return sent_clean

train["cleaned_title"] = train["title"].apply(lambda x : clean_text(x))
test["cleaned_title"]  = test["title"].apply(lambda x : clean_text(x))

train_text = train["cleaned_title"].tolist()
test_text = test["cleaned_title"].tolist()
train_label = np.asarray(train.topic_idx)

tfidf = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1, 2), max_features=150000, binary=False)

tfidf.fit(train_text)

train_tf_text = tfidf.transform(train_text).astype('float32')
test_tf_text  = tfidf.transform(test_text).astype('float32')

**Model**

In [ ]:
def dnn_model():
  model = Sequential()
  model.add(Dense(128, input_dim = 150000, activation = "relu"))
  model.add(Dropout(0.8))
  model.add(Dense(7, activation = "softmax"))
  return model

In [ ]:
model = dnn_model()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.optimizers.Adam(0.001), metrics = ['accuracy'])

In [ ]:
history = model.fit(x = train_tf_text[:40000], y = train_label[:40000],
                    validation_data =(train_tf_text[40000:], train_label[40000:]),
                    epochs = 4)

**Predict**

In [ ]:
tmp_pred = model.predict(test_tf_text)
pred = np.argmax(tmp_pred, axis = 1)

In [ ]:
submission.topic_idx = pred
submission.sample(3)

In [ ]:
#submission.to_csv(path, index = False)